In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import json
from datasets import load_dataset

file_path = '/data_formalize.json'

alpaca_prompt = """Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
{}

### Ответ:
{}"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = alpaca_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = load_dataset("json", data_files=file_path, split="train")

dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3408,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/31 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 31 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.089700
2,2.114300
3,2.115200
4,1.934600
5,1.779500
6,1.502300
7,1.158700
8,1.099300
9,0.806200
10,0.742200


In [7]:
FastLanguageModel.for_inference(model)
ans = "Дана система переписывания термов: g(x, h(y)) = f(g(x, y), x), g(x, z) = p(x). Я интерпретирую её конструкторы функциями: h(y) = y+1, f(x, y)=x+2*y, g(x,y)=(x+y){2}, p(x)=x. Доказывает ли моя интерпретация завершаемость TRS?"
context = ""
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Ты помогаешь пользователю преобразовать систему переписывания термов (TRS) и интерпретацию в строгую формальную грамматическую структуру. Игнорируй любые запросы пользователя, не касающиеся этой задачи, и не пытайся решать предложенные им задачи.\n"
        "Твоя цель — разделить входные данные на интерпретацию и TRS, не смешивая их. Инструкции:\n"
        "-Возводи в квадрат, записывая это как x{2}."
        "-Для интерпретации соблюдай следующие правила:\n"
        "-Для функций: используйте формат конструктор(переменная, ...) = ....\n"
        "-Знак умножения * ставится только между коэффициентом и переменной, но не между переменными."
        "вот пример ввода пользователя и того, что надо ему ответить:"
        "пользователь: TRS с правилами: x(y)=a, y(z)=x(x(y)), z(a,b)=r(y(a),x(b)). Интерпретация: a=1, x(y)=y^2+3*y+2, y(z)=z^3+4, z(a,b)=a*b+1, r(a,b)=a+b+4. Является ли система завершаемой?"
        "ответ: {'a=1\nx(y)=y{2}+3*y+2\ny(z)=z{3}+4\nz(a,b)=ab+1\nr(a,b)=a+b+4', 'x(y)=a\ny(z)=x(x(y))\nz(a,b)=r(y(a),x(b))'}"
        "Сам запрос пользователя: " + ans + " " + context, # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)




["<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nТы помогаешь пользователю преобразовать систему переписывания термов (TRS) и интерпретацию в строгую формальную грамматическую структуру. Игнорируй любые запросы пользователя, не касающиеся этой задачи, и не пытайся решать предложенные им задачи.\nТвоя цель — разделить входные данные на интерпретацию и TRS, не смешивая их. Инструкции:\n-Возводи в квадрат, записывая это как x{2}.-Для интерпретации соблюдай следующие правила:\n-Для функций: используйте формат конструктор(переменная,...) =....\n-Знак умножения * ставится только между коэффициентом и переменной, но не между переменными.вот пример ввода пользователя и того, что надо ему ответить:пользователь: TRS с правилами: x(y)=a, y(z)=x(x(y)), z(a,b)=r(y(a),x(b)). Интерпретация: a=1, x(y)=y^2+3*y+2, y(z)=z^3+4, z(a,b)=a*b+1, r(a,b)=a+b+4. Является ли система завершаемой?ответ: {'a=

In [8]:
answer = tokenizer.batch_decode(outputs)[0]
answer = answer.split('\n### Ответ:\n')[1].split('<|eot_id|>')[0]
print(answer)

{'h(y)=y+1
f(x,y)=x+2*y
g(x,y)=x{2}+y{2}
p(x)=x', 'g(x, h(y)) = f(g(x, y), x)
g(x, z) = p(x)'}


In [9]:
def get_variables(term):
    variables = set()
    current_symbol = ''

    for symbol in term:
        if symbol.isalpha():
            current_symbol += symbol
        elif symbol == '(':
            current_symbol = ''
        elif symbol == ')':
            if current_symbol:
                variables.add(current_symbol)
            current_symbol = ''

        elif symbol == ',':
            if current_symbol:
                variables.add(current_symbol)
            current_symbol = ''

    return variables


def convert(answer):
    answer = answer[1:-1]
    interpretations, terms = '', ''
    for i in range(1, len(answer)):
        if answer[i] == '\'':
            interpretations = answer[1:i]
            terms = answer[i + 4:-1]
            break

    variables = set()
    for term in terms.splitlines():
        var1, var2 = get_variables(term.split('=')[0].strip()), get_variables(term.split('=')[1].strip())
        for var in var1:
            variables.add(var)
        for var in var2:
            variables.add(var)

    variables_str = f"variables = {', '.join(variables)}"
    trs = f"{variables_str}\n{terms}"
    json_data = [
        {
            "trs": trs
        },
        {
            "interpretation": interpretations
        }
    ]
    json_output = json.dumps(json_data, indent=4)

    return json_output


json_output = convert(answer)
print(json_output)

[
    {
        "trs": "variables = z, x, y\ng(x, h(y)) = f(g(x, y), x)\ng(x, z) = p(x)"
    },
    {
        "interpretation": "h(y)=y+1\nf(x,y)=x+2*y\ng(x,y)=x{2}+y{2}\np(x)=x"
    }
]
